# LIAR DETECTION GROUP PROJECT

Run the cell below to import packages.

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
from functools import reduce

#import unittest
from IPython.display import display, HTML
#from sklearn.utils import shuffle
# NLTK for NLP utils and corpora
#import nltk
from collections import defaultdict
# NumPy and TensorFlow
import numpy as np
import pandas as pd
#import tensorflow as tf
import pickle
import dill
# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import timeit  #For timing
from sklearn.feature_extraction.text import CountVectorizer
#from project_files import pdio # for saving and loading dataframes

### Load data
Loading data from Pre-Processing step Part 1

In [2]:
#Read pkl file from part 1
# all_data = pd.read_pickle('parsed_data/df_alldata1.pkl')
all_data = pd.read_pickle('parsed_data/df_liarpolitifact_data_embed.pkl')
all_data.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23258 entries, 0 to 23257
Data columns (total 6 columns):
target            23258 non-null int64
title             23258 non-null object
title_tokcan      23258 non-null object
title_POS         23258 non-null object
binary_target     23258 non-null int64
embedded_title    23258 non-null object
dtypes: int64(2), object(4)
memory usage: 19.1 MB


In [3]:
#select column to train on 
# mini_data = all_data[:10]

tok_char_n = lambda a, ng_len: [a[i:i+ng_len] for i in range(len(a))][:-ng_len]

n_gram_len = 3
field_name='title'
new_field_name = field_name+'_pad'
tok_field_name = field_name+'_tok'
tok_int_field_name = field_name+'_tok_int'

# find 95% quantile
text_lens=[len(i) for i in all_data[field_name]]
pad_len = int(np.percentile(text_lens, 95))

# truncate/pad new field to 95th percentile
all_data[new_field_name] = [(" ".join(i.lower().split()) + pad_len * ' ')[:pad_len] for i in all_data[field_name]]

# tokenize
all_data[tok_field_name] = [tok_char_n(i,n_gram_len) for i in all_data[new_field_name]]

# Define helper functions
Helper functions for processing

In [4]:
# pulled from a StackOverflow post

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

# Process character n-grams
Create character n-grams for each of the news entries. This gives me the vocabulary.

In [8]:
# for ngram_len in range(start_ngram,start_ngram):
# vec = []
# # ngram = []
# # start_ngram = 2
# # vocab = []
# i = ngram_len-2
vec=CountVectorizer(analyzer='char', binary=True, ngram_range=(n_gram_len, n_gram_len))
ngram = vec.fit_transform(all_data[field_name][all_data.duplicated(['title'])==False])
vocab = vec.vocabulary_
vocab['  '] = len(vocab)
vocab['oov'] = len(vocab)
default = len(vocab)-n_gram_len

tok = lambda voc, toks: [voc.get(t,default) for t in toks]

all_data[tok_int_field_name] = [tok(vocab, j) for j in all_data[tok_field_name]]

all_data[tok_int_field_name][:10]
print(default)

12875


In [9]:
ttl = list(all_data['title'])
print(len(ttl))
ml = list(all_data[tok_int_field_name])
mask = [12878 in i for i in ml]
sum(mask)
# print(all_data['title','binary_target'].is_unique)
# print([i for i in range(len(mask)) if mask[i]])
# print(all_data.count())
j = all_data['title'].duplicated()
# j[:5]
print(j.count())

23258
23258


# Build and run Keras LSTM


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

all_data_train, all_data_dev, all_data_test = train_validate_test_split(all_data, train_percent = .7, validate_percent=.15)

x_train = np.array(list(all_data_train[tok_int_field_name]))
print("x_train: ", x_train.shape)
y_train = np.reshape(list(all_data_train['binary_target']),[-1,1])
print("y_train: ", y_train.shape)
x_test = np.array(list(all_data_dev[tok_int_field_name]))
print("x_test: ", x_test.shape)
y_test = np.reshape(list(all_data_dev['binary_target']),[-1,1])
print("y_test: ", y_test.shape)

max_features = len(vocab)

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(72))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)
score = model.evaluate(x_test, y_test, batch_size=32)


# define model
# model = Sequential()
# model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by Interac

x_train:  (16280, 189)
y_train:  (16280, 1)
x_test:  (3488, 189)
y_test:  (3488, 1)
Epoch 1/2
 1632/16280 [==>...........................] - ETA: 3:48 - loss: 13.8324 - acc: 0.3444

InvalidArgumentError: indices[9,12] = 12878 is not in [0, 12878)
	 [[Node: embedding_2/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_1/RMSprop/Assign_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_2/embeddings/read, embedding_2/Cast, training_1/RMSprop/gradients/embedding_2/embedding_lookup_grad/concat/axis)]]

In [ ]:
indices[15, 40]

list(all_data_dev[tok_int_field_name][:10])

In [ ]:
# x_train = np.array(list(all_data_train[tok_int_field_name]))
# print("x_train: ", x_train.shape)
# y_train = np.array(list(all_data_train['target']))
# print("y_train: ", y_train.shape)
# x_test = np.array(list(all_data_dev[tok_int_field_name]))
# print("x_test: ", x_test.shape)
# y_test = np.reshape(list(all_data_dev['target']),[1,-1])
# print("y_test: ", y_test.shape)

score = model.evaluate(x_test, y_test, batch_size=16)

# isot score

In [ ]:
score

In [ ]:
default

In [ ]:
all_data[]